This file takes the original dataset from Kaggle and converts the dataset into Python-readable values

In [5]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import math
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from tabulate import tabulate
import sys
sys.path.append('C:\\Users\\chery\\Documents\\SeattlePolice911Response')
from utils.baseML import BaseML
from datetime import datetime


Read in CSV

In [2]:
spir_original = pd.read_csv("Seattle_Police_Department_911_Incident_Response.csv")

C:\Users\chery\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
print(spir_original.columns)

Index(['CAD CDW ID', 'CAD Event Number', 'General Offense Number',
       'Event Clearance Code', 'Event Clearance Description',
       'Event Clearance SubGroup', 'Event Clearance Group',
       'Event Clearance Date', 'Hundred Block Location', 'District/Sector',
       'Zone/Beat', 'Census Tract', 'Longitude', 'Latitude',
       'Incident Location', 'Initial Type Description',
       'Initial Type Subgroup', 'Initial Type Group', 'At Scene Time'],
      dtype='object')


 Drop unused Columns

In [3]:
drop_columns= ['CAD CDW ID', 'CAD Event Number', 'General Offense Number', 'Hundred Block Location', 'District/Sector',
       'Zone/Beat', 'Longitude', 'Latitude',
       'Incident Location']

spir_drop_unused_col = spir_original.drop(labels=drop_columns, axis=1)

Create list of unique census tracts. This is used to pull data from the US Census API 5 year American Community Survey

In [13]:
census_tracts = spir_drop_unused_col["Census Tract"].unique()
census_tracts = pd.DataFrame(census_tracts, columns=["census_tracts"])
census_tracts.dropna(axis =0, inplace=True)
census_tracts.census_tracts=census_tracts.census_tracts.apply(format_tract)
census_tracts.census_tracts = pd.Series(census_tracts.census_tracts.unique())
census_tracts.dropna(axis=0, inplace=True)
census_tracts.to_csv("census_tracts.csv")

Fix date and time

In [ ]:
class Seattle_Police_ML(BaseML):

    def dayDifference(self, fdate, ldate):
        delta = ldate - fdate
        return(delta)

    def transform_times(self):
        self.df["at_scene_datetime"] = self.df.apply(lambda x: x.At_Scene_Date + "T" + x.At_Scene_Time, axis=1)
        self.df['at_scene_temp'] = self.df.at_scene_datetime.apply(lambda x: datetime.fromisoformat(x))
        self.df.Event_Clearance_Date.fillna("2021-05-13", inplace=True)
        self.df.Event_Clearance_Time.fillna("00:00:00", inplace=True)
        #self.df[["Event_Clearance_Date", "Event_Clearance_Time"]] = self.df[["Event_Clearance_Date", "Event_Clearance_Time"]].astype(str)
        #print(spir.df.Event_Clearance_Date.dtype)
        self.df["event_clear_datetime"] = self.df.apply(lambda x: x.Event_Clearance_Date + "T" + x.Event_Clearance_Time, axis=1)
        self.df['event_clear_temp'] = self.df.event_clear_datetime.apply(lambda x: datetime.fromisoformat(x))
        self.df['time_at_scene'] = self.df.apply(lambda x: pd.to_timedelta(self.dayDifference(x["at_scene_temp"], x["event_clear_temp"])), axis = 1)
        self.df.drop(["At_Scene_Date","At_Scene_Time", "Event_Clearance_Date", "Event_Clearance_Time"], axis = 1, inplace = True)
        #self.df.drop(self.df[self.df['Schedule_to_appt_days'] < 0].index, inplace = True)
        #self.df.dropna(inplace=True)
        print(self.df.sample(3))

    def simplify_times(self):
        bins = (
            pd.Timedelta(minutes = 0),
            pd.Timedelta(minutes = 30),
            pd.Timedelta(minutes = 60),
            pd.Timedelta(hours = 4),
            pd.Timedelta(hours = 8),
            pd.Timedelta(hours = 12),
            pd.Timedelta(days = 1),
            pd.Timedelta(days = 36500)
        )

        labels = ['< 30min', '30-60min','1-4hrs', '4-8hrs','8-12hrs','12-24hrs', '24hrs+']

        return pd.cut(self.df['time_at_scene'], bins, labels = labels)
    

    def encode_features(self):
        features = ['Fare', 'Cabin', 'Age', 'Sex', 'Lname', 'NamePrefix']
        df_combined = pd.concat([df_train[features], df_test[features]])
        
        for feature in features:
            le = preprocessing.LabelEncoder()
            le = le.fit(df_combined[feature])
            df_train[feature] = le.transform(df_train[feature])
            df_test[feature] = le.transform(df_test[feature])
        return df_train, df_test
        
        data_train, data_test = encode_features(data_train, data_test)
        data_train.head()

    def normalize_features(self):
        scaler = preprocessing.StandardScaler().fit(self.df)
        df_scaled = scaler.transform(self.df)
        self.df = pd.DataFrame(df_scaled, columns = self.df.columns, dtype= 'int64')


In [4]:
print(spir_drop_unused_col['Event Clearance Date'].sample(3))

721206    10/04/2016 11:08:47 AM
208989    10/07/2011 10:03:00 PM
571939    11/23/2012 07:12:00 AM
Name: Event Clearance Date, dtype: object


Isolate missing values